In [ ]:
# Version 1.2
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

import re #Module expressions régulières pour modifier certaines colones

## Partie 1: Ouverture et étude des données

### Ouverture et étude des données

In [ ]:
laptop_data = pd.read_csv(r'laptops.csv', sep=",", index_col=12, encoding='latin1') #J'ai modifié l'index pour la mettre sur le prix

In [ ]:
laptop_data.head()

In [ ]:
laptop_data.shape

In [ ]:
laptop_data.dtypes

In [ ]:
laptop_data.loc['499,00']

In [ ]:
laptop_data['ID'] = range(1, len(laptop_data) + 1)

In [ ]:
#Modifications sur le dataset afin de le rendre exploitable pour analyse

#On convertit les valeurs de la catégorie et de l'OS en chiffre

print((laptop_data['Category'].unique()))

laptop_data.Category[laptop_data.Category == 'Ultrabook'] = 1
laptop_data.Category[laptop_data.Category == 'Notebook'] = 2
laptop_data.Category[laptop_data.Category == 'Netbook'] = 3
laptop_data.Category[laptop_data.Category == 'Gaming'] = 4
laptop_data.Category[laptop_data.Category == '2 in 1 Convertible'] = 5
laptop_data.Category[laptop_data.Category == 'Workstation'] = 6

laptop_data.rename(columns = {'Operating System':'OperatingSystem'}, inplace = True)
print((laptop_data['OperatingSystem'].unique()))

laptop_data.OperatingSystem[laptop_data.OperatingSystem== 'macOS'] = 1
laptop_data.OperatingSystem[laptop_data.OperatingSystem== 'No OS'] = 2
laptop_data.OperatingSystem[laptop_data.OperatingSystem== 'Windows'] = 3
laptop_data.OperatingSystem[laptop_data.OperatingSystem== 'Mac OS'] = 4
laptop_data.OperatingSystem[laptop_data.OperatingSystem== 'Linux'] = 5
laptop_data.OperatingSystem[laptop_data.OperatingSystem== 'Android'] = 6
laptop_data.OperatingSystem[laptop_data.OperatingSystem== 'Chrome OS'] = 7

print((laptop_data['Manufacturer'].unique()))

laptop_data.Manufacturer[laptop_data.Manufacturer== 'Apple'] = 1
laptop_data.Manufacturer[laptop_data.Manufacturer== 'HP'] = 2
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Acer'] = 3
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Asus'] = 4
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Dell'] = 5
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Lenovo'] = 6
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Chuwi'] = 7
laptop_data.Manufacturer[laptop_data.Manufacturer== 'MSI'] = 8
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Microsoft'] = 9
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Toshiba'] = 10
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Huawei'] = 11
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Xiaomi'] = 12
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Vero'] = 13
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Razer'] = 14
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Mediacom'] = 15
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Samsung'] = 16
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Google'] = 17
laptop_data.Manufacturer[laptop_data.Manufacturer== 'Fujitsu'] = 18
laptop_data.Manufacturer[laptop_data.Manufacturer== 'LG'] = 19

In [ ]:
gpu_mapping = {}  # Dictionnaire pour mapper les fabricants aux numéros
gpu = laptop_data['GPU'].unique()
cpu_mapping = {}
cpu = laptop_data['CPU'].unique()

for i, gpu in enumerate(gpu, start=1):
    gpu_mapping[gpu] = i

for i, cpu in enumerate(cpu, start=1):
    cpu_mapping[cpu] = i

laptop_data['GPU'] = laptop_data['GPU'].map(gpu_mapping)

laptop_data['CPU'] = laptop_data['CPU'].map(cpu_mapping)


In [ ]:
#On supprime les colones inexploitables
laptop_data = laptop_data.drop('Model Name', axis=1)
laptop_data = laptop_data.drop('Operating System Version', axis=1)

#On supprime les unités à la fin de la colone Weight, Screen Size et RAM
laptop_data['Weight'] = laptop_data['Weight'].str.replace('kg', '').str.replace('s', '').astype(float)

laptop_data.rename(columns = {'Screen Size':'ScreenSize'}, inplace = True)
laptop_data['ScreenSize'] = laptop_data['ScreenSize'].str.replace('"', '').astype(float)

laptop_data['RAM'] = laptop_data['RAM'].str.replace('GB', '').str.replace('s', '').astype(float)

#Pour la colone ScreenSize, on supprime tout ce qu'il y a avant la taille avec re (module d'expressions régulières)
laptop_data['Screen'] = laptop_data['Screen'].apply(lambda x: re.findall('\d{4}x\d{4}|\d{4}x\d{3}', x)[0] if isinstance(x, str) else x)
#Ensuite on sépare la première taille de la deuxième
laptop_data['Screen'] = laptop_data['Screen'].str.split('x', n=1)
#Ensuite on fait la multiplication afin d'avoir un seul chiffre
laptop_data['Screen'] = laptop_data['Screen'].apply(lambda x: int(x[0]) * int(x[1]) if isinstance(x, list) and len(x) == 2 else x)

In [ ]:
print (laptop_data[' Storage'].unique())

In [ ]:
laptop_data[' StorageType'] = laptop_data[' Storage'].str.replace('.*GB', '', regex=True)
print (laptop_data[' StorageType'].unique())

In [ ]:
storagetype_mapping = {} 
storagetype = laptop_data[' StorageType'].unique()

for i, storage in enumerate(storagetype, start=1):
    storagetype_mapping[storage] = i

laptop_data[' StorageType'] = laptop_data[' StorageType'].map(storagetype_mapping)

In [ ]:
Storage_mapping = {
    '256GB SSD': 256,
    '1TB HDD': 1000,
    '500GB HDD': 500,
    '512GB SSD': 512,
    '128GB SSD +  1TB HDD': 128 + 1000,
    '128GB SSD': 128,
    '256GB SSD +  1TB HDD': 256 + 1000,
    '32GB Flash Storage': 32,
    '2TB HDD': 2000,
    '64GB Flash Storage': 64,
    '512GB SSD +  1TB HDD': 512 + 1000,
    '1TB SSD': 1000,
    '256GB SSD +  2TB HDD': 256 + 2000,
    '1TB Hybrid': 1000,
    '256GB Flash Storage': 256,
    '16GB Flash Storage': 16,
    '32GB SSD': 32,
    '1GB SSD': 1,
    '128GB Flash Storage': 128,
    '512GB SSD +  2TB HDD': 512 + 2000,
    '16GB SSD': 16,
    '128GB SSD +  2TB HDD': 128 + 2000,
    '1TB SSD +  1TB HDD': 1000 + 1000,
    '256GB SSD +  500GB HDD': 256 + 500,
    '512GB Flash Storage': 512,
    '256GB SSD +  256GB SSD': 256 + 256,
    '512GB SSD +  512GB SSD': 512 + 512,
    '8GB SSD': 8,
    '64GB Flash Storage +  1TB HDD': 64 + 1000,
    '256GB SSD +  1TB Hybrid': 256 + 1000,
    '128GB HDD': 128,
    '1TB HDD +  1TB HDD': 1000 + 1000,
    '240GB SSD': 240,
    '32GB HDD': 32,
    '64GB SSD': 64,
    '512GB SSD +  1TB Hybrid': 512 + 1000,
    '508GB Hybrid': 508,
    '512GB SSD +  256GB SSD': 512 + 256
}

laptop_data[' Storage'] = laptop_data[' Storage'].replace(Storage_mapping)

In [ ]:
#laptop_data['Weight'] = laptop_data['Weight'].astype(int)

In [ ]:
laptop_data.head(20)